In this notebook, we compare **Autoencoders** and **Principal Component Analysis (PCA)** as dimensionality reduction techniques on the Fashion-MNIST dataset.

The goal is to visualize, reconstruct, and cluster the data using both techniques and evaluate their quality using clustering metrics like:

- Adjusted Rand Index (ARI)
- Homogeneity Score
- Silhouette Score

We will also visualize the latent spaces using t-SNE.


# 1. Import and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, homogeneity_score, silhouette_score
from tensorflow import keras


# 2. Load and Preprocess MNIST-Fashion Dataset

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_train = X_train.reshape(-1, 28 * 28)
X_valid = X_valid.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

cluster_labels = [f"Cluster {i}" for i in range(10)]


# 3. Define Encoder and Decoder

In [ ]:
input_shape = X_train.shape[1]
encoder_output_shape = 100

encoder = keras.Sequential([
    keras.layers.InputLayer(input_shape),
    keras.layers.Dense(512),
    keras.layers.Dense(256),
    keras.layers.Dense(128),
    keras.layers.Dense(64),
    keras.layers.Dense(encoder_output_shape)
])

decoder = keras.Sequential([
    keras.layers.InputLayer((encoder_output_shape,)),
    keras.layers.Dense(64),
    keras.layers.Dense(128),
    keras.layers.Dense(256),
    keras.layers.Dense(512),
    keras.layers.Dense(input_shape)
])

autoencoder = keras.Sequential([encoder, decoder])
autoencoder.compile(loss=keras.losses.MeanSquaredError(),optimizer=keras.optimizers.Adam(learning_rate=0.001))

early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = autoencoder.fit(X_train, X_train,epochs=50,validation_data=(X_valid, X_valid),
                          callbacks=[early_stopping_cb],verbose=0)

pd.DataFrame(history.history).plot()
plt.title("Autoencoder Training Loss")
plt.grid()
plt.show()


# 4. Reconstruction Examples

In [ ]:
def plot_reconstructions(model, images=X_valid, n_images=5):
    reconstructions = np.clip(model.predict(images[:n_images]), 0, 1)
    fig = plt.figure(figsize=(n_images * 1.5, 3))

    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plt.imshow(images[image_index].reshape(28, 28), cmap="binary")
        plt.axis("off")
        if image_index == 0:
            plt.title("Original", fontsize=18, pad=10)

    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plt.imshow(reconstructions[image_index].reshape(28, 28), cmap="binary")
        plt.axis("off")
        if image_index == 0:
            plt.title("Reconstruction", fontsize=18, pad=10)
    plt.tight_layout()
    plt.show()

plot_reconstructions(autoencoder)


# 5. Visualization Utility

In [ ]:
def visualize_compressed(x_compressed, y, images, result='Result', class_names=None):
    plt.figure(figsize=(8, 6))
    cmap = plt.cm.tab10
    Z = (x_compressed - x_compressed.min()) / (x_compressed.max() - x_compressed.min())
    scatter = plt.scatter(Z[:, 0], Z[:, 1], c=y, s=10, cmap=cmap)

    image_positions = np.array([[1., 1.]])
    for index, position in enumerate(Z):
        dist = ((position - image_positions) ** 2).sum(axis=1)
        if dist.min() > 0.02:
            image_positions = np.r_[image_positions, [position]]
            imagebox = mpl.offsetbox.AnnotationBbox(
                mpl.offsetbox.OffsetImage(images[index].reshape(28,28), cmap="binary", zoom=0.6),
                position,
                bboxprops={"edgecolor": cmap(y[index]), "lw": 1}
            )
            plt.gca().add_artist(imagebox)

    if class_names:
        handles = [plt.Line2D([0], [0], marker='o', color='w', label=class_names[i],
                              markerfacecolor=cmap(i), markersize=6) for i in np.unique(y)]
        plt.legend(handles=handles, loc='best', fontsize=10)

    plt.axis("off")
    plt.title(f"{result}: 2D Latent Space", fontsize=14)
    plt.tight_layout()
    plt.show()


## Apply PCA and t-SNE

In [ ]:
pca = PCA(n_components=encoder_output_shape).fit(X_train)
X_valid_pca = pca.transform(X_valid)
X_valid_pca_tsne = TSNE(n_components=2, random_state=42).fit_transform(X_valid_pca)

X_valid_ae = encoder.predict(X_valid)
X_valid_ae_tsne = TSNE(n_components=2, random_state=42).fit_transform(X_valid_ae)


# Clustering and Metrics

In [ ]:
kmeans_ae = KMeans(n_clusters=10, random_state=42, n_init='auto').fit(X_valid_ae)
kmeans_pca = KMeans(n_clusters=10, random_state=42, n_init='auto').fit(X_valid_pca)

labels_ae = kmeans_ae.fit_predict(X_valid_aed)
labels_pca = kmeans_pca.fit_predict(X_valid_pca)

ari_ae = adjusted_rand_score(y_valid, kmeans_ae.labels_)
ari_pca = adjusted_rand_score(y_valid, kmeans_pca.labels_)

hom_ae = homogeneity_score(y_valid, kmeans_ae.labels_)
hom_pca = homogeneity_score(y_valid, kmeans_pca.labels_)

sil_ae = silhouette_score(X_valid_ae, kmeans_ae.labels_)
sil_pca = silhouette_score(X_valid_pca, kmeans_pca.labels_)

print(f"ARI (Autoencoder): {ari_ae:.3f}")
print(f"ARI (PCA): {ari_pca:.3f}")
print(f"Homogeneity (Autoencoder): {hom_ae:.3f}")
print(f"Homogeneity (PCA): {hom_pca:.3f}")
print(f"Silhouette (Autoencoder): {sil_ae:.3f}")
print(f"Silhouette (PCA): {sil_pca:.3f}")


# Visualize t-SNE Embeddings

In [ ]:
visualize_compressed(X_valid_pca_tsne, y_valid, X_valid, result='PCA t-SNE', class_names=class_names)
visualize_compressed(X_valid_ae_tsne, y_valid, X_valid, result='Autoencoder t-SNE', class_names=class_names)

visualize_compressed(X_valid_aed_tsne, labels_ae, X_valid, result='Autoencoder KMeans Clustering',class_names=cluster_labels)
visualize_compressed(X_valid_pca_tsne, labels_pca, X_valid, result='PCA KMeans Clustering',class_names=cluster_labels)


###  Observations

- The **Autoencoder** achieves slightly better **Homogeneity** and **Silhouette** scores than PCA.
- However, **PCA** is still competitive and often faster to compute.
- Reconstruction quality is significantly better with the autoencoder due to its non-linear capacity.

###  Theoretical Insight

> If the autoencoder uses only **linear activations** and the cost function is **MSE**, it ends up performing **PCA** (as shown in Chapter 8 of Hands-On ML).

Since our autoencoder uses non-linear activations, it can **learn more complex manifolds** and outperform PCA in certain settings, especially for reconstruction tasks.

###  Final Thought

Use **PCA** when you want speed and interpretability.

Use **Autoencoders** when you want non-linear compression, better reconstructions, and can afford extra training time.
